In [520]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [521]:
#Cargar desde un archivo .csv sin indice
#df = pd.read_csv('Albany.csv')
#df.head(5)


df = pd.read_csv('Datos_limpios_Mexico.csv')
df =df.drop(labels=["Unnamed: 0"], axis=1)
df

last_scraped           source  \
0         25/09/24      city scrape   
1         26/09/24  previous scrape   
2         26/09/24      city scrape   
3         25/09/24      city scrape   
4         25/09/24      city scrape   
...            ...              ...   
26577     25/09/24      city scrape   
26578     25/09/24      city scrape   
26579     25/09/24      city scrape   
26580     25/09/24      city scrape   
26581     25/09/24      city scrape   

                                                   name  \
0                                           Villa Dante   
1                                          Condesa Haus   
2                     2 bedroom apt. deco bldg, Condesa   
3      Beautiful light Studio Coyoacan- full equipped !   
4                                    NEW  DESIGNER LOFT   
...                                                 ...   
26577    Charming 2-Bedroom Retreat in Lomas de Bezares   
26578                     Suite King | District Polanco   
26579                     Suite King | District Polanco   
26580                    Suite Queen | District Polanco   
26581                  Pretty, well centered, amenities   

                                             description   host_name  \
0      Dentro de Villa un estudio de arte con futon, ...        Dici   
1      A new concept of hosting in mexico through a b...    Fernando   
2      Comfortably furnished, sunny, 2 bedroom apt., ...    Nicholas   
3      COYOACAN designer studio quiet & safe! well eq...      Trisha   
4      Is the best ever place triple L <br />Location...      Andrea   
...                                                  ...         ...   
26577  Experience Mexico City in this delightful retr...     HostPal   
26578  This spacious room features a plush king-size ...  Luis Angel   
26579  Discover the essence of luxury in our king-siz...  Luis Angel   
26580  Welcome to your refuge in the heart of Polanco...  Luis Angel   
26581  Known as New Polanco, the Granada colony has p...    Salvador   

      host_since        host_location  \
0       28/06/10  Mexico City, Mexico   
1       09/08/10  Mexico City, Mexico   
2       04/01/11  Mexico City, Mexico   
3       24/08/10  Mexico City, Mexico   
4       27/04/11  Mexico City, Mexico   
...          ...                  ...   
26577   16/04/16  Mexico City, Mexico   
26578   13/08/24  Mexico City, Mexico   
26579   13/08/24  Mexico City, Mexico   
26580   13/08/24  Mexico City, Mexico   
26581   22/05/13  Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Master in visual arts, film photography & Mark...  a few days or more   
1      Condesa Haus  offers independent  studios and ...      within an hour   
2      I am a journalist writing about food, (book an...      within an hour   
3      I am a mother, documentary film maker and phot...  within a few hours   
4      I Leave in Mexico City... I am an Architect an...      within an hour   
...                                                  ...                 ...   
26577  I am a social person, who likes to provide the...      within an hour   
26578  We offer beautiful, comfortable spaces that wo...      within an hour   
26579  We offer beautiful, comfortable spaces that wo...      within an hour   
26580  We offer beautiful, comfortable spaces that wo...      within an hour   
26581                                                  .      within an hour   

      host_response_rate  ... number_of_reviews_l30d review_scores_rating  \
0                     0%  ...                    0.0                 4.80   
1                   100%  ...                    0.0                 4.58   
2                    71%  ...                    0.0                 4.90   
3                   100%  ...                    2.0                 4.91   
4                   100%  ...                    0.0                 4.91   
...                  ...  ...                  

**Convertimos a numericos**

In [522]:
df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float) / 100
df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [523]:
df['property_type'] = df['property_type'].replace({
    'Entire rental unit': 1,
    'Entire condo': 2,
    'Private room in rental unit': 3,
    'Entire home': 4,
    'Private room in condo': 5,
    'Entire townhouse': 6,
    'Private room in home': 7,
    'Entire villa': 8,
    'Entire loft': 9,
    'Private room in villa': 10,
    'Entire serviced apartment': 11,
    'Private room in townhouse': 12,
    'Room in hotel': 13,
    'Entire guesthouse': 14,
    'Private room in bed and breakfast': 15,
    'Room in aparthotel': 16,
    'Private room in guest suite': 17,
    'Private room in casa particular': 18,
    'Shared room in hotel': 19,
    'Private room in guesthouse': 20,
    'Tiny home': 21,
    'Shared room in hostel': 22,
    'Private room in serviced apartment': 23,
    'Private room in boat': 24,
    'Houseboat': 25,
    'Entire guest suite': 26,
    'Boat': 27,
    'Shared room in rental unit': 28,
    'Shared room in condo': 29,
    'Private room in loft': 30,
    'Private room': 31,
    'Private room in bungalow': 32,
    'Private room in hostel': 33,
    'Entire bungalow': 34,
    'Entire vacation home': 35,
    'Entire cabin': 36,
    'Private room in shipping container': 37,
    'Room in hostel': 38,
    'Camper/RV': 39,
    'Entire place': 40,
    'Barn': 41,
    'Casa particular': 42,
    'Entire cottage': 43,
    'Private room in hut': 44,
    'Shared room in bungalow': 45,
    'Private room in tiny home': 46,
    'Shared room in bed and breakfast': 47,
    'Private room in vacation home': 48,
    'Hut': 49,
    'Private room in barn': 50,
    'Minsu': 51,
    'Shipping container': 52,
    'Private room in cabin': 53,
    'Tower': 54
})


**Reemplazar valores**

_host_acceptance_rate_

In [524]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array([0.  , 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.13, 0.14, 0.15, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
       0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35,
       0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46,
       0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57,
       0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68,
       0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79,
       0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9 ,
       0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  ])

In [525]:
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100

In [526]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 1 if isinstance(x, (int, float)) and x > 0.80 else (0 if isinstance(x, (int, float)) and x <= 0.80 else 'Sin respuesta')
)

df

last_scraped           source  \
0         25/09/24      city scrape   
1         26/09/24  previous scrape   
2         26/09/24      city scrape   
3         25/09/24      city scrape   
4         25/09/24      city scrape   
...            ...              ...   
26577     25/09/24      city scrape   
26578     25/09/24      city scrape   
26579     25/09/24      city scrape   
26580     25/09/24      city scrape   
26581     25/09/24      city scrape   

                                                   name  \
0                                           Villa Dante   
1                                          Condesa Haus   
2                     2 bedroom apt. deco bldg, Condesa   
3      Beautiful light Studio Coyoacan- full equipped !   
4                                    NEW  DESIGNER LOFT   
...                                                 ...   
26577    Charming 2-Bedroom Retreat in Lomas de Bezares   
26578                     Suite King | District Polanco   
26579                     Suite King | District Polanco   
26580                    Suite Queen | District Polanco   
26581                  Pretty, well centered, amenities   

                                             description   host_name  \
0      Dentro de Villa un estudio de arte con futon, ...        Dici   
1      A new concept of hosting in mexico through a b...    Fernando   
2      Comfortably furnished, sunny, 2 bedroom apt., ...    Nicholas   
3      COYOACAN designer studio quiet & safe! well eq...      Trisha   
4      Is the best ever place triple L <br />Location...      Andrea   
...                                                  ...         ...   
26577  Experience Mexico City in this delightful retr...     HostPal   
26578  This spacious room features a plush king-size ...  Luis Angel   
26579  Discover the essence of luxury in our king-siz...  Luis Angel   
26580  Welcome to your refuge in the heart of Polanco...  Luis Angel   
26581  Known as New Polanco, the Granada colony has p...    Salvador   

      host_since        host_location  \
0       28/06/10  Mexico City, Mexico   
1       09/08/10  Mexico City, Mexico   
2       04/01/11  Mexico City, Mexico   
3       24/08/10  Mexico City, Mexico   
4       27/04/11  Mexico City, Mexico   
...          ...                  ...   
26577   16/04/16  Mexico City, Mexico   
26578   13/08/24  Mexico City, Mexico   
26579   13/08/24  Mexico City, Mexico   
26580   13/08/24  Mexico City, Mexico   
26581   22/05/13  Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Master in visual arts, film photography & Mark...                   1   
1      Condesa Haus  offers independent  studios and ...                   2   
2      I am a journalist writing about food, (book an...                   2   
3      I am a mother, documentary film maker and phot...                   3   
4      I Leave in Mexico City... I am an Architect an...                   2   
...                                                  ...                 ...   
26577  I am a social person, who likes to provide the...                   2   
26578  We offer beautiful, comfortable spaces that wo...                   2   
26579  We offer beautiful, comfortable spaces that wo...                   2   
26580  We offer beautiful, comfortable spaces that wo...                   2   
26581                                                  .                   2   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    0.00  ...                     0.0                  4.80   
1                    1.00  ...                     0.0                  4.58   
2                    0.71  ...                     0.0                  4.90   
3                    1.00  ...                     2.0                  4.91   
4                    1.00  ...                     0.0                  4.91   
...                   ...  ..

_room_type_

In [527]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array([1, 2, 3, 4])

In [528]:
#Convierto una variable a dicotómica
#df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], 0)
#df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)
#df

_availability_365_

In [529]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [530]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 1 if x == 365 else (0 if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

last_scraped           source  \
0         25/09/24      city scrape   
1         26/09/24  previous scrape   
2         26/09/24      city scrape   
3         25/09/24      city scrape   
4         25/09/24      city scrape   
...            ...              ...   
26577     25/09/24      city scrape   
26578     25/09/24      city scrape   
26579     25/09/24      city scrape   
26580     25/09/24      city scrape   
26581     25/09/24      city scrape   

                                                   name  \
0                                           Villa Dante   
1                                          Condesa Haus   
2                     2 bedroom apt. deco bldg, Condesa   
3      Beautiful light Studio Coyoacan- full equipped !   
4                                    NEW  DESIGNER LOFT   
...                                                 ...   
26577    Charming 2-Bedroom Retreat in Lomas de Bezares   
26578                     Suite King | District Polanco   
26579                     Suite King | District Polanco   
26580                    Suite Queen | District Polanco   
26581                  Pretty, well centered, amenities   

                                             description   host_name  \
0      Dentro de Villa un estudio de arte con futon, ...        Dici   
1      A new concept of hosting in mexico through a b...    Fernando   
2      Comfortably furnished, sunny, 2 bedroom apt., ...    Nicholas   
3      COYOACAN designer studio quiet & safe! well eq...      Trisha   
4      Is the best ever place triple L <br />Location...      Andrea   
...                                                  ...         ...   
26577  Experience Mexico City in this delightful retr...     HostPal   
26578  This spacious room features a plush king-size ...  Luis Angel   
26579  Discover the essence of luxury in our king-siz...  Luis Angel   
26580  Welcome to your refuge in the heart of Polanco...  Luis Angel   
26581  Known as New Polanco, the Granada colony has p...    Salvador   

      host_since        host_location  \
0       28/06/10  Mexico City, Mexico   
1       09/08/10  Mexico City, Mexico   
2       04/01/11  Mexico City, Mexico   
3       24/08/10  Mexico City, Mexico   
4       27/04/11  Mexico City, Mexico   
...          ...                  ...   
26577   16/04/16  Mexico City, Mexico   
26578   13/08/24  Mexico City, Mexico   
26579   13/08/24  Mexico City, Mexico   
26580   13/08/24  Mexico City, Mexico   
26581   22/05/13  Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Master in visual arts, film photography & Mark...                   1   
1      Condesa Haus  offers independent  studios and ...                   2   
2      I am a journalist writing about food, (book an...                   2   
3      I am a mother, documentary film maker and phot...                   3   
4      I Leave in Mexico City... I am an Architect an...                   2   
...                                                  ...                 ...   
26577  I am a social person, who likes to provide the...                   2   
26578  We offer beautiful, comfortable spaces that wo...                   2   
26579  We offer beautiful, comfortable spaces that wo...                   2   
26580  We offer beautiful, comfortable spaces that wo...                   2   
26581                                                  .                   2   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    0.00  ...                     0.0                  4.80   
1                    1.00  ...                     0.0                  4.58   
2                    0.71  ...                     0.0                  4.90   
3                    1.00  ...                     2.0                  4.91   
4                    1.00  ...                     0.0                  4.91   
...                   ...  ..

_host_response_time_

In [531]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array([1, 2, 3, 4])

In [532]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more", "unknown"], 0)
df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)
df

last_scraped           source  \
0         25/09/24      city scrape   
1         26/09/24  previous scrape   
2         26/09/24      city scrape   
3         25/09/24      city scrape   
4         25/09/24      city scrape   
...            ...              ...   
26577     25/09/24      city scrape   
26578     25/09/24      city scrape   
26579     25/09/24      city scrape   
26580     25/09/24      city scrape   
26581     25/09/24      city scrape   

                                                   name  \
0                                           Villa Dante   
1                                          Condesa Haus   
2                     2 bedroom apt. deco bldg, Condesa   
3      Beautiful light Studio Coyoacan- full equipped !   
4                                    NEW  DESIGNER LOFT   
...                                                 ...   
26577    Charming 2-Bedroom Retreat in Lomas de Bezares   
26578                     Suite King | District Polanco   
26579                     Suite King | District Polanco   
26580                    Suite Queen | District Polanco   
26581                  Pretty, well centered, amenities   

                                             description   host_name  \
0      Dentro de Villa un estudio de arte con futon, ...        Dici   
1      A new concept of hosting in mexico through a b...    Fernando   
2      Comfortably furnished, sunny, 2 bedroom apt., ...    Nicholas   
3      COYOACAN designer studio quiet & safe! well eq...      Trisha   
4      Is the best ever place triple L <br />Location...      Andrea   
...                                                  ...         ...   
26577  Experience Mexico City in this delightful retr...     HostPal   
26578  This spacious room features a plush king-size ...  Luis Angel   
26579  Discover the essence of luxury in our king-siz...  Luis Angel   
26580  Welcome to your refuge in the heart of Polanco...  Luis Angel   
26581  Known as New Polanco, the Granada colony has p...    Salvador   

      host_since        host_location  \
0       28/06/10  Mexico City, Mexico   
1       09/08/10  Mexico City, Mexico   
2       04/01/11  Mexico City, Mexico   
3       24/08/10  Mexico City, Mexico   
4       27/04/11  Mexico City, Mexico   
...          ...                  ...   
26577   16/04/16  Mexico City, Mexico   
26578   13/08/24  Mexico City, Mexico   
26579   13/08/24  Mexico City, Mexico   
26580   13/08/24  Mexico City, Mexico   
26581   22/05/13  Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Master in visual arts, film photography & Mark...                   1   
1      Condesa Haus  offers independent  studios and ...                   2   
2      I am a journalist writing about food, (book an...                   2   
3      I am a mother, documentary film maker and phot...                   3   
4      I Leave in Mexico City... I am an Architect an...                   2   
...                                                  ...                 ...   
26577  I am a social person, who likes to provide the...                   2   
26578  We offer beautiful, comfortable spaces that wo...                   2   
26579  We offer beautiful, comfortable spaces that wo...                   2   
26580  We offer beautiful, comfortable spaces that wo...                   2   
26581                                                  .                   2   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    0.00  ...                     0.0                  4.80   
1                    1.00  ...                     0.0                  4.58   
2                    0.71  ...                     0.0                  4.90   
3                    1.00  ...                     2.0                  4.91   
4                    1.00  ...                     0.0                  4.91   
...                   ...  ..

_accommodates_

In [533]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([1. , 2. , 2.9, 3. , 4. , 5. , 6. , 7. ])

In [534]:
df['accommodates'] = df['accommodates'].replace([1, 2, 2.9], 0).replace([3, 4, 5, 6, 7], 1)
df

last_scraped           source  \
0         25/09/24      city scrape   
1         26/09/24  previous scrape   
2         26/09/24      city scrape   
3         25/09/24      city scrape   
4         25/09/24      city scrape   
...            ...              ...   
26577     25/09/24      city scrape   
26578     25/09/24      city scrape   
26579     25/09/24      city scrape   
26580     25/09/24      city scrape   
26581     25/09/24      city scrape   

                                                   name  \
0                                           Villa Dante   
1                                          Condesa Haus   
2                     2 bedroom apt. deco bldg, Condesa   
3      Beautiful light Studio Coyoacan- full equipped !   
4                                    NEW  DESIGNER LOFT   
...                                                 ...   
26577    Charming 2-Bedroom Retreat in Lomas de Bezares   
26578                     Suite King | District Polanco   
26579                     Suite King | District Polanco   
26580                    Suite Queen | District Polanco   
26581                  Pretty, well centered, amenities   

                                             description   host_name  \
0      Dentro de Villa un estudio de arte con futon, ...        Dici   
1      A new concept of hosting in mexico through a b...    Fernando   
2      Comfortably furnished, sunny, 2 bedroom apt., ...    Nicholas   
3      COYOACAN designer studio quiet & safe! well eq...      Trisha   
4      Is the best ever place triple L <br />Location...      Andrea   
...                                                  ...         ...   
26577  Experience Mexico City in this delightful retr...     HostPal   
26578  This spacious room features a plush king-size ...  Luis Angel   
26579  Discover the essence of luxury in our king-siz...  Luis Angel   
26580  Welcome to your refuge in the heart of Polanco...  Luis Angel   
26581  Known as New Polanco, the Granada colony has p...    Salvador   

      host_since        host_location  \
0       28/06/10  Mexico City, Mexico   
1       09/08/10  Mexico City, Mexico   
2       04/01/11  Mexico City, Mexico   
3       24/08/10  Mexico City, Mexico   
4       27/04/11  Mexico City, Mexico   
...          ...                  ...   
26577   16/04/16  Mexico City, Mexico   
26578   13/08/24  Mexico City, Mexico   
26579   13/08/24  Mexico City, Mexico   
26580   13/08/24  Mexico City, Mexico   
26581   22/05/13  Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Master in visual arts, film photography & Mark...                   1   
1      Condesa Haus  offers independent  studios and ...                   2   
2      I am a journalist writing about food, (book an...                   2   
3      I am a mother, documentary film maker and phot...                   3   
4      I Leave in Mexico City... I am an Architect an...                   2   
...                                                  ...                 ...   
26577  I am a social person, who likes to provide the...                   2   
26578  We offer beautiful, comfortable spaces that wo...                   2   
26579  We offer beautiful, comfortable spaces that wo...                   2   
26580  We offer beautiful, comfortable spaces that wo...                   2   
26581                                                  .                   2   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    0.00  ...                     0.0                  4.80   
1                    1.00  ...                     0.0                  4.58   
2                    0.71  ...                     0.0                  4.90   
3                    1.00  ...                     2.0                  4.91   
4                    1.00  ...                     0.0                  4.91   
...                   ...  ..

In [535]:
df['availability_365']

0        0
1        0
2        0
3        0
4        0
        ..
26577    1
26578    0
26579    0
26580    0
26581    0
Name: availability_365, Length: 26582, dtype: int64

**REGRESIÓN LOGÍSTICA**

_host_is_superhost_

In [536]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_acceptance_rate', 'host_response_rate', 'number_of_reviews_ltm']]
Var_Dep = df['host_is_superhost']

In [537]:
X= Vars_Indep
y= Var_Dep

In [538]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

In [539]:
escalar =StandardScaler()

In [540]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [541]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [542]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [543]:
y_pred =algoritma.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [544]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[8755 1315]
 [3630 2250]]


In [545]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6311360448807855


In [546]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6899686520376176


In [547]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8694141012909633


_host_identity_verified_

In [548]:
#Declaramos las variables dependientes e independientes para regresión logistica
Vars_Indep=df[['has_availability', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['host_identity_verified']

In [549]:
df['host_acceptance_rate'].dtype

dtype('int64')

In [550]:
X1= Vars_Indep
y1= Var_Dep

In [551]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [552]:
escalar =StandardScaler()

In [553]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [554]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [555]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [556]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [557]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 253   94]
 [  60 7568]]


In [558]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9877316627512399


In [559]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9806896551724138


In [560]:
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.729106628242075


_host_acceptance_rate_

In [561]:
Vars_Indep= df[["accommodates", "room_type","instant_bookable"]]
Var_Dep= df["host_acceptance_rate"]

In [562]:
X= Vars_Indep
y= Var_Dep

In [563]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None)

In [564]:
escalar= StandardScaler()

In [565]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [566]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [567]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [568]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [569]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[   1 1088]
 [   9 6877]]


In [570]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.8634023854362838


In [571]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.8624451410658307


In [572]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.0009182736455463728


_has_availability_

In [573]:
Vars_Indep=df[['host_identity_verified', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['has_availability']

In [574]:
df['host_identity_verified'].dtype

dtype('int64')

In [575]:
#Redefinimos las variables
X3 = Vars_Indep
y3 = Var_Dep

In [576]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=None)

In [577]:
escalar =StandardScaler()

In [578]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [579]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [580]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [581]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [582]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 155   59]
 [  79 5024]]


In [583]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9883926814873106


In [584]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9740455143878127


In [585]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label= 0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7242990654205608


_instant_bookable_

In [586]:
Vars_Indep= df[["accommodates", "room_type"]]
Var_Dep= df["instant_bookable"]

In [587]:
X= Vars_Indep
y= Var_Dep

In [588]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [589]:
escalar= StandardScaler()

In [590]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [591]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [592]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [593]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [594]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[4845    0]
 [3130    0]]


In [595]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [596]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.6075235109717868


In [597]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_room_type_

In [598]:
Vars_Indep= df[["accommodates", "room_type", "instant_bookable"]]
Var_Dep= df["room_type"]

In [599]:
X= Vars_Indep
y= Var_Dep

In [600]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [601]:
escalar= StandardScaler()

In [602]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [603]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [604]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [605]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 3, ..., 1, 2, 1])

In [606]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[5174    0    0    0]
 [   0 2652    0    0]
 [   0    0  111    0]
 [   0    0    0   38]]


In [607]:
from sklearn.metrics import precision_score

#precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
#print("predicciones del modelo")
#print(precision)



precision= precision_score(y_test, y_pred, average="weighted")
print("predicciones del modelo")
print(precision)

predicciones del modelo
1.0


In [608]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
1.0


In [609]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="weighted")
print("Sensibilidad del modelo: ")
print(sensibilidad)




Sensibilidad del modelo: 
1.0


_accommodates_

In [610]:
Vars_Indep= df[[ "room_type", "instant_bookable"]]
Var_Dep= df["accommodates"]

In [611]:
X= Vars_Indep
y= Var_Dep

In [612]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [613]:
escalar= StandardScaler()

In [614]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [615]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [616]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [617]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0., 1., 1., ..., 1., 0., 1.])

In [618]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[2393 2048]
 [ 325 3209]]


In [619]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)



predicciones del modelo
0.6104241963096824


In [620]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7024451410658307


In [621]:
from sklearn.metrics import recall_score

#sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
#print("Sensibilidad del modelo: ")
#print(sensibilidad)

sensibilidad= recall_score(y_test, y_pred, average="weighted")
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.7024451410658307


_availability_365_

In [622]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_response_rate','price', 'room_type', 'review_scores_location', 'host_identity_verified']]
Var_Dep = df['availability_365']

In [623]:
df['review_scores_cleanliness'].dtype

dtype('float64')

In [624]:
df['host_response_rate'].dtype

dtype('float64')

In [625]:
df['price'].dtype

dtype('float64')

In [626]:
df['review_scores_location'].dtype

dtype('float64')

In [627]:
df['property_type'].dtype

dtype('O')

In [628]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [629]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [630]:
#Se escalan todos los datos
escalar = StandardScaler()

In [631]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)




In [632]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [633]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [634]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [635]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[7609    1]
 [ 365    0]]


In [636]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


In [637]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9541065830721003


In [638]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9998685939553219


_host_response_time_

In [639]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'host_total_listings_count', 'host_response_rate']]
Var_Dep = df['host_response_time']

In [640]:
df['host_response_time'].dtype
df['host_response_time'].drop_duplicates().tolist()

[1, 2, 3, 4]

In [641]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [642]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [643]:
#Se escalan todos los datos
escalar = StandardScaler()

In [644]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [645]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [646]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [647]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 2, 4, ..., 2, 1, 4])

In [648]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 327    0    0    3]
 [   0 5334    5  266]
 [   0  463    0  113]
 [   0  535    0  929]]


In [649]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="weighted")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7635113206257046


In [650]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.826332288401254


In [651]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="weighted")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.826332288401254


_host_has_profile_pic_

In [652]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'review_scores_rating', 'host_total_listings_count', 'host_is_superhost']]
Var_Dep = df['host_has_profile_pic']

In [653]:
df['review_scores_rating'].dtype

dtype('float64')

In [654]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [655]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [656]:
#Se escalan todos los datos
escalar = StandardScaler()

In [657]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [658]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [659]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [660]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [661]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   2  151]
 [   5 7817]]


In [662]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9810491967871486


In [663]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9804388714733542


In [664]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.013071895424836602
